In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import yfinance as yf
from textblob import TextBlob
import tweepy

# Step 1: Fetch Historical Stock Data using Yahoo Finance API
def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data[['Close']]

# Example: Fetching Google stock data
stock_data = fetch_stock_data('GOOG', '2020-01-01', '2023-01-01')

# Step 2: Preprocessing Stock Data
def preprocess_stock_data(data, time_window):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(time_window, len(scaled_data)):
        X.append(scaled_data[i-time_window:i, 0])
        y.append(scaled_data[i, 0])

    return np.array(X), np.array(y), scaler

time_window = 60
X_stock, y_stock, scaler = preprocess_stock_data(stock_data.values, time_window)
X_stock = np.reshape(X_stock, (X_stock.shape[0], X_stock.shape[1], 1))

# Step 3: Fetch Tweets and Perform Sentiment Analysis using TextBlob
def fetch_tweets(api_key, api_secret_key, access_token, access_token_secret, query):
    auth = tweepy.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    tweets = tweepy.Cursor(api.search_tweets, q=query, lang="en").items(100)
    sentiments = []

    for tweet in tweets:
        analysis = TextBlob(tweet.text)
        sentiments.append(analysis.sentiment.polarity)

    return np.mean(sentiments)

# Example: Replace with your Twitter API credentials and query
api_key = "your_api_key"
api_secret_key = "your_api_secret_key"
access_token = "your_access_token"
access_token_secret = "your_access_token_secret"
sentiment_score = fetch_tweets(api_key, api_secret_key, access_token, access_token_secret, "Google")

# Integrate sentiment score into the dataset (optional step if combining features)
sentiment_feature = np.full((len(y_stock), 1), sentiment_score)
X_combined = np.concatenate([X_stock[:, :, 0], sentiment_feature], axis=1).reshape(X_stock.shape[0], X_stock.shape[1], -1)

# Step 4: Build Multi-Layer Bidirectional LSTM Model with TensorFlow/Keras
model = Sequential([
    Bidirectional(LSTM(50, return_sequences=True), input_shape=(X_combined.shape[1], X_combined.shape[2])),
    Dropout(0.2),
    Bidirectional(LSTM(50)),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Step 5: Train the Model with Early Stopping and Dropout Regularization
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(X_combined, y_stock,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping])

# Step 6: Evaluate the Model Performance
y_pred_scaled = model.predict(X_combined)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_actual = scaler.inverse_transform(y_stock.reshape(-1, 1))

mae = mean_absolute_error(y_actual, y_pred)
correlation = np.corrcoef(y_actual.flatten(), y_pred.flatten())[0][1]

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Correlation Coefficient: {correlation}")

# Step 7: Visualize Predictions vs Actual Stock Prices
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(y_actual.flatten(), label="Actual Prices", color='blue')
plt.plot(y_pred.flatten(), label="Predicted Prices", color='red')
plt.title("Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend()
plt.show()